In [1]:
import os
import subprocess
import sagemaker
import tensorflow as tf
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()
role = get_execution_role()

In [2]:
# !pygmentize 'source/core/step_three_recognize_process/tools/train/

#single gpu training
from sagemaker.tensorflow import TensorFlow

# Can be local_gpu, local, or ml.p3.2xlarge, ml.p3.8xlarge etc.
instance_type='ml.p3.2xlarge'
#bucket="<your-s3-bucket-name>"
bucket="uwo-bkt-xj"
prefix="audio-denoise"

#inputs="s3://sagemaker-us-east-1-002224604296/denoise/tfrecords/"
input_path='s3://{}/{}/'.format(bucket, prefix)
hyperparameters = {'servable_model_dir': '/opt/ml/model', 
                   'epoch': 1, 
                   'task_type': 'train', 
                   'sagemaker': 'true'
                  }
# requirements.txt are not allowed in script mode (BYOS). https://stackoverflow.com/q/53530867/3252127
# Hyperparameters are specified in the model_config.py file, we need to extract them out to use SageMaker automl features.
segan_estimator = TensorFlow(entry_point='source/main_train_deploy.py',
                             source_dir='.',
                             model_dir='/opt/ml/model',
                             role=role,
                             output_path='s3://{}/{}/output'.format(bucket, prefix),
                             framework_version='1.15.2',
                             hyperparameters=hyperparameters, #set weights_path if you are finetune a pretrained model
                             py_version='py2',
                             train_instance_type=instance_type,
                             train_instance_count=1)

2.1.0 is the latest version of tensorflow that supports Python 2. Newer versions of tensorflow will only be available for Python 3.Please set the argument "py_version='py3'" to use the Python 3 tensorflow image.
Legacy mode is deprecated in versions 1.13 and higher. Using script mode instead.


In [3]:
print(input_path)

s3://uwo-bkt-xj/audio-denoise/


In [4]:
result = segan_estimator.fit(input_path)

2020-06-25 14:53:28 Starting - Starting the training job...
2020-06-25 14:53:30 Starting - Launching requested ML instances......
2020-06-25 14:54:44 Starting - Preparing the instances for training......
2020-06-25 14:55:48 Downloading - Downloading input data............
2020-06-25 14:57:46 Training - Downloading the training image...
2020-06-25 14:58:07 Training - Training image download completed. Training in progress.WARNING:tensorflow:From /usr/local/lib/python2.7/dist-packages/tensorflow_core/__init__.py:1473: The name tf.estimator.inputs is deprecated. Please use tf.compat.v1.estimator.inputs instead.

2020-06-25 14:58:11,502 sagemaker-containers INFO     Imported framework sagemaker_tensorflow_container.training
2020-06-25 14:58:12,161 sagemaker-containers INFO     Invoking user script

Training Env:

{
    "module_dir": "s3://uwo-bkt-xj/tensorflow-training-2020-06-25-14-53-28-492/source/sourcedir.tar.gz", 
    "channel_input_dirs": {
        "training": "/opt/ml/input/data/tra

In [5]:
predictor=segan_estimator.deploy(initial_instance_count=1, 
                                 instance_type='ml.g4dn.xlarge', 
                                 endpoint_type='tensorflow-serving')

-------------!

In [6]:
from scipy.io import wavfile
import tensorflow as tf
import numpy as np
import time

def de_emph(y, coeff=0.95):
    if coeff <= 0:
        return y
    x = np.zeros(y.shape[0], dtype=np.float32)
    x[0] = y[0]
    for n in range(1, y.shape[0], 1):
        x[n] = coeff * x[n - 1] + y[n]
    return x

def pre_emph_np(x, coeff=0.95):
    x0 = np.reshape(x[0], [1,])
    diff = x[1:] - coeff * x[:-1]
#     concat = tf.concat(0, [x0, diff])
    concat = np.concatenate([x0, diff], 0)
    return concat

def pre_emph_test_np(coeff, wav):
    x_preemph = pre_emph_np(wav, coeff)
    return x_preemph
    
def clean_serving(x, canvas_size, preemph, predictor=None):
    """ clean a utterance x
        x: numpy array containing the normalized noisy waveform
    """
    c_res = None
    print('start timer')
    Time1 = time.time()
    for beg_i in range(0, x.shape[0], canvas_size):
        if x.shape[0] - beg_i  < canvas_size:
            length = x.shape[0] - beg_i
            pad = canvas_size - length
        else:
            length = canvas_size
            pad = 0
        x_ = np.zeros((1, canvas_size))
        print(x_[0].shape)
        if pad > 0:
            x_ = np.concatenate((x[beg_i:beg_i + length], np.zeros(pad)))
        else:
            x_ = x[beg_i:beg_i + length]
        print('Cleaning chunk {} -> {}'.format(beg_i, beg_i + length))
        canvas_w = None
        # fdict = {self.gtruth_noisy[0]:x_}
        # canvas_w = self.sess.run(self.Gs[0],
        #                         feed_dict=fdict)[0]
        test_example = {'wav_and_noisy': x_.tolist()}
        if predictor == None:
            canvas_w = x_
        else:
            canvas_w = predictor.predict(test_example)
        canvas_w = np.reshape(np.array(canvas_w['predictions']),canvas_size)
        print('canvas w shape: ', canvas_w.shape)
        if pad > 0:
            print('Removing padding of {} samples'.format(pad))
            # get rid of last padded samples
            canvas_w = canvas_w[:-pad]
        if c_res is None:
            c_res = canvas_w
        else:
            c_res = np.concatenate((c_res, canvas_w))
    # deemphasize
    c_res = de_emph(c_res, preemph)
    print('finish {}'.format(time.time()-Time1))
    return c_res

In [7]:
#test_wav = "../../record_noise/record_noise_28_clip_0.wav"
test_wav = "../test/record_noise_28_clip_0.wav"
preemph = 0.95
canvas_size = 2**14

fm, wav_data = wavfile.read(test_wav)
wavname = test_wav.split('/')[-1]
wave = (2./65535.) * (wav_data.astype(np.float32) - 32767) + 1.

x_pholder_np = pre_emph_test_np(preemph, wave)

c_wave = clean_serving(x=x_pholder_np, canvas_size=canvas_size, preemph=preemph, predictor=predictor)

wavfile.write(os.path.join('../result', wavname), 16000, c_wave)

start timer
(16384,)
Cleaning chunk 0 -> 16384
canvas w shape:  (16384,)
(16384,)
Cleaning chunk 16384 -> 32768
canvas w shape:  (16384,)
(16384,)
Cleaning chunk 32768 -> 49152
canvas w shape:  (16384,)
(16384,)
Cleaning chunk 49152 -> 65536
canvas w shape:  (16384,)
(16384,)
Cleaning chunk 65536 -> 81920
canvas w shape:  (16384,)
(16384,)
Cleaning chunk 81920 -> 98304
canvas w shape:  (16384,)
(16384,)
Cleaning chunk 98304 -> 114688
canvas w shape:  (16384,)
(16384,)
Cleaning chunk 114688 -> 131072
canvas w shape:  (16384,)
(16384,)
Cleaning chunk 131072 -> 147456
canvas w shape:  (16384,)
(16384,)
Cleaning chunk 147456 -> 163840
canvas w shape:  (16384,)
(16384,)
Cleaning chunk 163840 -> 180224
canvas w shape:  (16384,)
(16384,)
Cleaning chunk 180224 -> 196608
canvas w shape:  (16384,)
(16384,)
Cleaning chunk 196608 -> 212992
canvas w shape:  (16384,)
(16384,)
Cleaning chunk 212992 -> 229376
canvas w shape:  (16384,)
(16384,)
Cleaning chunk 229376 -> 245760
canvas w shape:  (16384,)
